__качаем объявления о продаже недвижимости с avito.ru__

_Evgeny S. Borisov <parser@mechanoid.su>_

In [1]:
import logging
import sys 

logging.basicConfig(
        format=u'[%(levelname)-8s] %(asctime)s | %(message)s',
        datefmt='%Y-%m-%d %H:%M:%S',
        level=logging.INFO,
        # level=logging.DEBUG,
        stream=sys.stdout,
    )

In [2]:
from time import sleep
from tqdm.auto import tqdm
import pandas as pd
from datetime import datetime 

pd.set_option('display.max_colwidth', None)
pd.set_option('display.float_format', '{:.2f}'.format)
tqdm.pandas()

In [3]:
source = [
    {
            'place': 'sevastopol',
        'subfolder': 'flat',
              'url': 'https://www.avito.ru/sevastopol/kvartiry/prodam',
    },{
            'place': 'sevastopol',
        'subfolder': 'land',
              'url': 'https://www.avito.ru/sevastopol/zemelnye_uchastki/prodam',
    },{
            'place': 'sevastopol',
        'subfolder': 'house',
              'url': 'https://www.avito.ru/sevastopol/doma_dachi_kottedzhi/prodam',
    },
    {
            'place': 'sudak',
        'subfolder': 'flat',
              'url': 'https://www.avito.ru/sudak/kvartiry/prodam',
    },
    {
            'place': 'sudak',
        'subfolder': 'land',
              'url': 'https://www.avito.ru/sudak/zemelnye_uchastki/prodam',
    },{
            'place': 'sudak',
        'subfolder': 'house',
              'url': 'https://www.avito.ru/sudak/doma_dachi_kottedzhi/prodam',
    },
#     {
#             'place': 'bahchisaray',
#         'subfolder': 'flat',
#               'url': 'https://www.avito.ru/bahchisaray/kvartiry/prodam',
#     }, 
    {
            'place': 'bahchisaray',
        'subfolder': 'land',
              'url': 'https://www.avito.ru/bahchisaray/zemelnye_uchastki/prodam',
    },{
            'place': 'bahchisaray',
        'subfolder': 'house',
              'url': 'https://www.avito.ru/bahchisaray/doma_dachi_kottedzhi/prodam',
    },

]

---

In [4]:
# from selenium import webdriver
# from lib.avito import AvitoPageParser

# browser = webdriver.Firefox()
# browser.get(source[0]['url'])
# data = AvitoPageParser().transform(browser)
# # browser.close()

In [5]:
# browser.close()

In [6]:
# data

---

In [7]:
from selenium import webdriver
# from selenium.webdriver.common.by import By
from lib.avito import AvitoParser

In [8]:
dest_folder = 'data/2024/raw'

In [9]:
%%time 


options = webdriver.FirefoxOptions()
#options.headless = True
browser = webdriver.Firefox(options=options)
logging.info('open virtual browser')

for s in source:

    url = s['url']
    place = s['place']
    subfolder = s['subfolder'] 
    
    display(url)
    browser.get(url)
    # sleep(1)
    
    data = AvitoParser().transform(browser)
    assert len(data)>0
    
    df = pd.DataFrame(data)
    df['ts'] = datetime.now()
    df['place'] = place

    display( len(df) )
    t_now = datetime.now().strftime('%Y-%m-%d_%H-%M')
    fname = f'{dest_folder}/{subfolder}/avito/avito_{t_now}_raw_{place}.pkl'
    df.to_pickle(fname)
    display( fname )

browser.close()

[INFO    ] 2024-06-19 12:07:43 | open virtual browser


'https://www.avito.ru/sevastopol/kvartiry/prodam'

[INFO    ] 2024-06-19 12:07:54 | page 1: 53 items
[INFO    ] 2024-06-19 12:08:05 | page 2: 50 items
[INFO    ] 2024-06-19 12:08:16 | page 3: 50 items
[INFO    ] 2024-06-19 12:08:27 | page 4: 50 items
[INFO    ] 2024-06-19 12:08:36 | page 5: 50 items
[INFO    ] 2024-06-19 12:08:46 | page 6: 50 items
[INFO    ] 2024-06-19 12:08:55 | page 7: 50 items
[INFO    ] 2024-06-19 12:09:05 | page 8: 50 items
[INFO    ] 2024-06-19 12:09:15 | page 9: 50 items
[INFO    ] 2024-06-19 12:09:25 | page 10: 50 items
[INFO    ] 2024-06-19 12:09:35 | page 11: 50 items
[INFO    ] 2024-06-19 12:09:45 | page 12: 50 items
[INFO    ] 2024-06-19 12:09:56 | page 13: 50 items
[INFO    ] 2024-06-19 12:10:07 | page 14: 50 items
[INFO    ] 2024-06-19 12:10:18 | page 15: 50 items
[INFO    ] 2024-06-19 12:10:28 | page 16: 50 items
[INFO    ] 2024-06-19 12:10:39 | page 17: 50 items
[INFO    ] 2024-06-19 12:10:49 | page 18: 50 items
[INFO    ] 2024-06-19 12:10:59 | page 19: 50 items
[INFO    ] 2024-06-19 12:11:09 | page 20

4107

'data/2024/raw/flat/avito/avito_2024-06-19_12-22_raw_sevastopol.pkl'

...

'https://www.avito.ru/bahchisaray/doma_dachi_kottedzhi/prodam'

[INFO    ] 2024-06-19 12:41:22 | page 1: 50 items
[INFO    ] 2024-06-19 12:41:31 | page 2: 50 items
[INFO    ] 2024-06-19 12:41:39 | page 3: 50 items
[INFO    ] 2024-06-19 12:41:47 | page 4: 50 items
[INFO    ] 2024-06-19 12:41:56 | page 5: 50 items
[INFO    ] 2024-06-19 12:42:01 | page 6: 27 items


277

'data/2024/raw/house/avito/avito_2024-06-19_12-42_raw_bahchisaray.pkl'

CPU times: user 4min 27s, sys: 10.8 s, total: 4min 38s
Wall time: 34min 20s


In [10]:
# browser.close()

In [11]:
!date

Ср 19 июн 2024 12:42:01 MSK
